# Projeto 1 - Ciência dos Dados

Nome: Augusto Rocha Ribeiro


Nome: Lucas Franco Florentino

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [169]:
#-------------------------------------Importando as bibliotecas necessárias-----------------------------------------------------

import pandas as pd
import os
import re 
from nltk.tokenize import TweetTokenizer

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [170]:
#----------------------------------Pegando o excel com as classificações manuais dos tweets-------------------------------------

filename = 'Havaianas.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Havaianas.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [171]:
#----------------------------------------Definindo Relevantes e Irrelevantes----------------------------------------------------

train = pd.read_excel(filename)
train_relevante = train[train['Classificação']==1]
train_irrelevante = train[train['Classificação']==0]

test = pd.read_excel(filename, sheet_name = 'Teste')
test_relevante = test[test['Classificação']==1]
test_irrelevante = test[test['Classificação']==0]


#------------------------------------------Classifica as categorias-------------------------------------------------------------

train['Classificação'] = train['Classificação'].astype('category')
test['Classificação'] = test['Classificação'].astype('category')

train.Classificação.cat.categories = ['Irrelevante', 'Relevante']
test.Classificação.cat.categories = ['Irrelevante', 'Relevante']

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Foi considerado relevante os comentários que exaltavam o produto e/ou que deixavam implícita uma qualidade sobre o produto.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [172]:
#------------------------------------Função para retirada de caracteres indesejados---------------------------------------------

def cleanup(text):
    punctuation = '[!-.:?;@/""“”,_]'
    pattern = re.compile(punctuation)
    text_pronto = re.sub(pattern, '', text)
    return text_pronto


#-------------------------------------Cria uma string com todas as palavras e caracteres----------------------------------------

tweets_train = ''
for i in range(len(train)):
    tweets_train += train['Treinamento'].loc[i]

lista_train = tweet_tokenizer.tokenize(cleanup(tweets_train.lower()))  # Deixa a string somente com letras minúsculas,
                                                                       # aplica a função definida acima, separa os emojis
                                                                       # das palavras, e cria uma lista em que cada
                                                                       # palavra ou emoji é um elemento.

tabela_t = pd.Series(lista_train).value_counts()       # Faz uma tabela com as frequências de cada palavra na lista acima

In [173]:
#------------------------------------------Selecionando tweets relevantes-------------------------------------------------------

linhas_train_relev = train['Classificação'] == 'Relevante'
df_train_relev = train.loc[linhas_train_relev, :]

linhas_train_irrelev = train['Classificação'] == 'Irrelevante'
df_train_irrelev = train.loc[linhas_train_irrelev, :]

#------------------------------------------Juntando todos os tweets numa string-------------------------------------------------

tweets_train_relev = ''
for i in df_train_relev['Treinamento']:
    tweets_train_relev += i

tweets_train_irrelev = ''
for i in df_train_irrelev['Treinamento']:
    tweets_train_irrelev += i

    
#----------------------------------------Tornando as strings em listas de palavras----------------------------------------------

lista_train_relev = tweet_tokenizer.tokenize(cleanup(tweets_train_relev.lower()))

lista_train_irrelev = tweet_tokenizer.tokenize(cleanup(tweets_train_irrelev.lower()))

In [174]:
#-----------------------------------------Frequências para treinamento relevante------------------------------------------------

tabela_tra = pd.Series(lista_train_relev).value_counts()
tabela_trr = pd.Series(lista_train_relev).value_counts(True)

print('Frequência ABSOLUTA das palavras dos tweets RELEVANTES\n')
print(tabela_tra)

print('\n\nFrequência RELATIVA das palavras dos tweets RELEVANTES\n')
print(tabela_trr)   # tabela printada à título de consulta, pois não será usada posteriormente


#----------------------------------------Frequências para treinamento irrelevante-----------------------------------------------

tabela_tia = pd.Series(lista_train_irrelev).value_counts()
tabela_tir = pd.Series(lista_train_irrelev).value_counts(True)

print('\n\nFrequência ABSOLUTA das palavras dos tweets IRRELEVANTES\n')
print(tabela_tia)

print('\n\nFrequência RELATIVA das palavras dos tweets IRRELEVANTES\n')
print(tabela_tir)   # tabela printada à título de consulta, pois não será usada posteriormente

Frequência ABSOLUTA das palavras dos tweets RELEVANTES

havaianas               51
de                      32
que                     23
o                       18
e                       16
                        ..
4599976                  1
brankinha                1
httpstcowjirx            1
😔                        1
kkkkkkkkkkkkkkkkkkkk     1
Length: 418, dtype: int64


Frequência RELATIVA das palavras dos tweets RELEVANTES

havaianas               0.062119
de                      0.038977
que                     0.028015
o                       0.021924
e                       0.019488
                          ...   
4599976                 0.001218
brankinha               0.001218
httpstcowjirx           0.001218
😔                       0.001218
kkkkkkkkkkkkkkkkkkkk    0.001218
Length: 418, dtype: float64


Frequência ABSOLUTA das palavras dos tweets IRRELEVANTES

havaianas    193
de           131
e             86
uma           62
a             62
            ... 
estaria   

In [175]:
#--------------------------------Limpando os tweets e listando suas palavras linha a linha--------------------------------------

for i in range(len(test['Teste'])):
    test['Teste'][i] = tweet_tokenizer.tokenize(cleanup(test['Teste'][i].lower()))
    
test

C:\Users\augus\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Teste,Classificação
0,"[joshnaku, delicia, perfeito, minecraft, chine...",Irrelevante
1,"[vendese, foto, dos, meus, pés, feios, com, ma...",Relevante
2,"[rt, ribeirod, 10, não, vejo, a, hora, de, che...",Relevante
3,"[estou, indo, de, havaiana, eu, tem, problema,...",Relevante
4,"[menina, com, shorts, saia, e, havaianas, 🥰, 🥰...",Irrelevante
...,...,...
195,"[egirao, aeciodepapelao, mas, a, tv, é, uma, u...",Irrelevante
196,"[todo, mundo, q, for, vir, no, meu, aniversári...",Relevante
197,"[eu, prometi, pra, mim, mesmo, que, eu, nao, i...",Relevante
198,"[eu, sempre, tive, toque, com, havaianas, bran...",Irrelevante


In [176]:
def probabilidade(tweet, tabela, lista):     # Função para realizar o cálculo de probabilidades das palavras
    prob = 1
    for palavra in tweet:
        if not palavra in tabela:           # Se a palavra de teste não consta em treinamento, então de acordo com a suavização
            prob = prob / (len(lista) + len(tabela_t))  # de LaPlace, a probabilidade dessa palavra fica com numerador 1
        else:
            prob = prob * (tabela['%s'%(palavra)] + 1) / (len(lista) + len(tabela_t))
    return prob

def classificador(tweet):   # Compara a probabilidade de ser relevante com não relevante e devolve a resposta
    if probabilidade(tweet, tabela_tra, lista_train_relev) > probabilidade(tweet, tabela_tia, lista_train_irrelev):
        return 'Relevante'
    else:
        return 'Irrelevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [177]:
#-------------------------------------Adicionando uma nova coluna ao dataframe Teste--------------------------------------------

test['Performance'] = 'NaN'
for i in range(len(test)):
    test.loc[i, 'Performance'] = classificador(test.loc[i, 'Teste'])
test

,Teste,Classificação,Performance
0,"[joshnaku, delicia, perfeito, minecraft, chine...",Irrelevante,Relevante
1,"[vendese, foto, dos, meus, pés, feios, com, ma...",Relevante,Relevante
2,"[rt, ribeirod, 10, não, vejo, a, hora, de, che...",Relevante,Irrelevante
3,"[estou, indo, de, havaiana, eu, tem, problema,...",Relevante,Irrelevante
4,"[menina, com, shorts, saia, e, havaianas, 🥰, 🥰...",Irrelevante,Irrelevante
...,...,...,...
195,"[egirao, aeciodepapelao, mas, a, tv, é, uma, u...",Irrelevante,Relevante
196,"[todo, mundo, q, for, vir, no, meu, aniversári...",Relevante,Irrelevante
197,"[eu, prometi, pra, mim, mesmo, que, eu, nao, i...",Relevante,Irrelevante
198,"[eu, sempre, tive, toque, com, havaianas, bran...",Irrelevante,Irrelevante


In [178]:
#-------------------------------------Contando o percentual de acertos----------------------------------------------------------

contador = 0
for i in range(len(test)):
    if test.loc[i, 'Classificação'] == test.loc[i, 'Performance']:
        contador += 1
contador = contador / len(test)
    
contador

0.48

___
### Concluindo

In [185]:
#------------------------Dashboard para análise de sentimento e visualização de dados-------------------------------------------

quadro = pd.crosstab(test['Classificação'], test['Performance'], margins = True, normalize = True)
quadro

Performance,Irrelevante,Relevante,All
Classificação,,,
Irrelevante,0.45,0.35,0.8
Relevante,0.17,0.03,0.2
All,0.62,0.38,1.0


## Diferentes cenários:

* Classificação de artigos em um jornal, indicando o tema do artigo por meio do uso de jargões e o tipo de reportagem. 
* Classificação de e-mails, onde o classificador poderia indentificar e-mails e separa-lo como spam ou email importantes, dividindo-os e facilitando a vida do usuário.
* Classificação de artigos científicos em materias, indentificando as palavras chave de cada materia. Um exemplo seria um texto com a palavra "integral", onde, com os filtros certos, seria indentificada como artigo de matemática ou física.


## Melhorias:
* Aplicar filtros para descartar palavras que não serão úteis para o classificador, como conectivos e artigos.
* Aplicar uma função que classifique as frases em frases relevantes negativas e positivas, além de separar palavras que contenham sarcasmo indentificando palavras negativas na mesma frase.
* Aplicar mais categorias para melhorar a eficiência do classificador.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**